<a href="https://colab.research.google.com/github/geekpascal/DocumentSearchAgent/blob/main/DocumentSearchAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install chromadb sentence-transformers langchain datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s

In [2]:
import os
import numpy as np
import pandas as pd
from datasets import load_dataset
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import time

In [3]:
dataset = load_dataset("wikipedia", "20220301.en", split="train[:1000]")
print(f"Loaded {len(dataset)} Wikipedia articles")


documents = []
for i, article in enumerate(dataset):
   doc = {
       "id": f"doc_{i}",
       "title": article["title"],
       "text": article["text"],
       "url": article["url"]
   }
   documents.append(doc)


df = pd.DataFrame(documents)
df.head(3)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

wikipedia.py:   0%|          | 0.00/36.7k [00:00<?, ?B/s]

The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train-00000-of-00041.parquet:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

...

train-00039-of-00041.parquet:   0%|          | 0.00/192M [00:00<?, ?B/s]

train-00040-of-00041.parquet:   0%|          | 0.00/185M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6458670 [00:00<?, ? examples/s]

Loaded 1000 Wikipedia articles


,id,title,text,url
0,doc_0,Anarchism,Anarchism is a political philosophy and moveme...,https://en.wikipedia.org/wiki/Anarchism
1,doc_1,Autism,Autism is a neurodevelopmental disorder charac...,https://en.wikipedia.org/wiki/Autism
2,doc_2,Albedo,Albedo (; ) is the measure of the diffuse refl...,https://en.wikipedia.org/wiki/Albedo


In [4]:
text_splitter = RecursiveCharacterTextSplitter(
   chunk_size=1000,
   chunk_overlap=200,
   length_function=len,
)


chunks = []
chunk_ids = []
chunk_sources = []


for i, doc in enumerate(documents):
   doc_chunks = text_splitter.split_text(doc["text"])
   chunks.extend(doc_chunks)
   chunk_ids.extend([f"chunk_{i}_{j}" for j in range(len(doc_chunks))])
   chunk_sources.extend([doc["title"]] * len(doc_chunks))


print(f"Created {len(chunks)} chunks from {len(documents)} documents")

Created 32974 chunks from 1000 documents


In [5]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(model_name)


sample_text = "This is a sample text to test our embedding model."
sample_embedding = embedding_model.encode(sample_text)
print(f"Embedding dimension: {len(sample_embedding)}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding dimension: 384


In [6]:
chroma_client = chromadb.Client()


embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_name)


collection = chroma_client.create_collection(
   name="document_search",
   embedding_function=embedding_function
)


batch_size = 100
for i in range(0, len(chunks), batch_size):
   end_idx = min(i + batch_size, len(chunks))

   batch_ids = chunk_ids[i:end_idx]
   batch_chunks = chunks[i:end_idx]
   batch_sources = chunk_sources[i:end_idx]

   collection.add(
       ids=batch_ids,
       documents=batch_chunks,
       metadatas=[{"source": source} for source in batch_sources]
   )

   print(f"Added batch {i//batch_size + 1}/{(len(chunks)-1)//batch_size + 1} to the collection")


print(f"Total documents in collection: {collection.count()}")

Added batch 1/330 to the collection
Added batch 2/330 to the collection
Added batch 3/330 to the collection
Added batch 4/330 to the collection
Added batch 5/330 to the collection
Added batch 6/330 to the collection
Added batch 7/330 to the collection
Added batch 8/330 to the collection
Added batch 9/330 to the collection
Added batch 10/330 to the collection
Added batch 11/330 to the collection
Added batch 12/330 to the collection
Added batch 13/330 to the collection
Added batch 14/330 to the collection
Added batch 15/330 to the collection
Added batch 16/330 to the collection
Added batch 17/330 to the collection
Added batch 18/330 to the collection
Added batch 19/330 to the collection
Added batch 20/330 to the collection
Added batch 21/330 to the collection
Added batch 22/330 to the collection
Added batch 23/330 to the collection
Added batch 24/330 to the collection
Added batch 25/330 to the collection
Added batch 26/330 to the collection
Added batch 27/330 to the collection
Added batc

In [7]:
def search_documents(query, n_results=5):
   """
   Search for documents similar to the query.

   Args:
       query (str): The search query
       n_results (int): Number of results to return

   Returns:
       dict: Search results
   """
   start_time = time.time()

   results = collection.query(
       query_texts=[query],
       n_results=n_results
   )

   end_time = time.time()
   search_time = end_time - start_time

   print(f"Search completed in {search_time:.4f} seconds")
   return results


queries = [
   "What are the effects of climate change?",
   "History of artificial intelligence",
   "Space exploration missions"
]


for query in queries:
   print(f"\nQuery: {query}")
   results = search_documents(query)

   for i, (doc, metadata) in enumerate(zip(results['documents'][0], results['metadatas'][0])):
       print(f"\nResult {i+1} from {metadata['source']}:")
       print(f"{doc[:200]}...")


Query: What are the effects of climate change?
Search completed in 0.0327 seconds

Result 1 from Agriculture:
Climate change 

Climate change and agriculture are interrelated on a global scale. Global warming affects agriculture through changes in average temperatures, rainfall, and weather extremes (like sto...

Result 2 from Alps:
Scientists have been studying the impact of climate change and water use. For example, each year more water is diverted from rivers for snowmaking in the ski resorts, the effect of which is yet unknow...

Result 3 from Adelaide:
Climate...

Result 4 from Athens:
Climate...

Result 5 from Aberdeen:
Climate...

Query: History of artificial intelligence
Search completed in 0.0200 seconds

Result 1 from Artificial intelligence:
When access to digital computers became possible in the mid-1950s, AI research began to explore the possibility that human intelligence could be reduced to step-by-step symbol manipulation, known as S...

Result 2 from Artificial intell

In [8]:
def interactive_search():
   """
   Interactive search interface for the document search engine.
   """
   while True:
       query = input("\nEnter your search query (or 'quit' to exit): ")

       if query.lower() == 'quit':
           print("Exiting search interface...")
           break

       n_results = int(input("How many results would you like? "))

       results = search_documents(query, n_results)

       print(f"\nFound {len(results['documents'][0])} results for '{query}':")

       for i, (doc, metadata, distance) in enumerate(zip(
           results['documents'][0],
           results['metadatas'][0],
           results['distances'][0]
       )):
           relevance = 1 - distance
           print(f"\n--- Result {i+1} ---")
           print(f"Source: {metadata['source']}")
           print(f"Relevance: {relevance:.2f}")
           print(f"Excerpt: {doc[:300]}...")
           print("-" * 50)


interactive_search()


Enter your search query (or 'quit' to exit): what is artificial intelligence
How many results would you like? 3
Search completed in 0.0284 seconds

Found 3 results for 'what is artificial intelligence':

--- Result 1 ---
Source: Artificial intelligence
Relevance: 0.61
Excerpt: Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to natural intelligence displayed by animals including humans. Leading AI textbooks define the field as the study of "intelligent agents": any system that perceives its environment and takes actions that maximize its c...
--------------------------------------------------

--- Result 2 ---
Source: Artificial intelligence
Relevance: 0.26
Excerpt: The various sub-fields of AI research are centered around particular goals and the use of particular tools. The traditional goals of AI research include reasoning, knowledge representation, planning, learning, natural language processing, perception, and the ability to move and manipulate o

In [9]:
def filtered_search(query, filter_source=None, n_results=5):
   """
   Search with optional filtering by source.

   Args:
       query (str): The search query
       filter_source (str): Optional source to filter by
       n_results (int): Number of results to return

   Returns:
       dict: Search results
   """
   where_clause = {"source": filter_source} if filter_source else None

   results = collection.query(
       query_texts=[query],
       n_results=n_results,
       where=where_clause
   )

   return results


unique_sources = list(set(chunk_sources))
print(f"Available sources for filtering: {len(unique_sources)}")
print(unique_sources[:5])


if len(unique_sources) > 0:
   filter_source = unique_sources[0]
   query = "main concepts and principles"

   print(f"\nFiltered search for '{query}' in source '{filter_source}':")
   results = filtered_search(query, filter_source=filter_source)

   for i, doc in enumerate(results['documents'][0]):
       print(f"\nResult {i+1}:")
       print(f"{doc[:200]}...")

Available sources for filtering: 1000
['Anatole France', 'Anchor', 'Atlantis', 'Agnosticism', 'Aurangzeb']

Filtered search for 'main concepts and principles' in source 'Anatole France':

Result 1:
des Anges (Revolt of the Angels, 1914) is often considered Anatole France's most profound and ironic novel. Loosely based on the Christian understanding of the War in Heaven, it tells the story of Arc...

Result 2:
Clio (1900)
 Histoire comique (A Mummer's Tale) (1903)
 Sur la pierre blanche (The White Stone) (1905)
 L'Affaire Crainquebille (1901)
 L'Île des Pingouins (Penguin Island) (1908)
 Les Contes de Jacqu...

Result 3:
Prose fiction
 Jocaste et le chat maigre (Jocasta and the Famished Cat) (1879)
 Le Crime de Sylvestre Bonnard (The Crime of Sylvestre Bonnard) (1881)
 Les Désirs de Jean Servien (The Aspirations of Je...

Result 4:
Literary career
France began his literary career as a poet and a journalist. In 1869, Le Parnasse Contemporain published one of his poems, "". In 1875, he sa